<a href="https://colab.research.google.com/github/okeefey/MIDIGenerator/blob/main/MIDIGenerator_using_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [48]:

from music21 import *
import glob
from tqdm import tqdm
import numpy as np
import random
from tensorflow.keras.layers import LSTM,Dense,Input,Dropout
from tensorflow.keras.models import Sequential,Model,load_model
from sklearn.model_selection import train_test_split



# Reading MIDI files


# Reading notes

In [49]:
from google.colab import drive
drive.mount('/content/drive')

def read_files(file):
    notes = []
    try:
        midi = converter.parse(file)
        notes_to_parse = midi.flat.getElementsByClass([note.Note, chord.Chord])

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                chord_notes = '.'.join(str(pitch) for pitch in element.pitches)
                notes.append(chord_notes)
    except Exception as e:
        print(f"Error processing {file}: {e}")
    return notes  # Returns a list (possibly empty for invalid files)

all_files = glob.glob('/content/drive/MyDrive/All Midi Files/**/*.mid', recursive=True)  # Updated path
print(f"Found {len(all_files)} files.")

# Use a Python list to store variable-length sequences
notes_list = [read_files(file) for file in tqdm(all_files, position=0, leave=True)]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 93 files.


100%|██████████| 93/93 [00:02<00:00, 38.86it/s]


In [50]:
# Unique notes/chords across all files
notess = sum(notes_list, [])  # Flatten the list of lists
unique_notes = list(set(notess))
print("Unique Notes:", len(unique_notes))

# Calculate frequency using Counter
from collections import Counter
freq = Counter(notess)

for i in range(10, 50, 10):
    count = sum(1 for cnt in freq.values() if cnt >= i)
    print(f"Notes with frequency >= {i}: {count}")

Unique Notes: 557
Notes with frequency >= 10: 40
Notes with frequency >= 20: 23
Notes with frequency >= 30: 18
Notes with frequency >= 40: 10


### Threshold

In [51]:
#filter notes greater than threshold i.e. 5
freq_notes=dict(filter(lambda x:x[1]>=5,freq.items()))

#create new notes using the frequent notes
new_notes=[[i for i in j if i in freq_notes] for j in notes_list]

In [52]:
#dictionary having key as note index and value as note
ind2note=dict(enumerate(freq_notes))

#dictionary having key as note and value as note index
note2ind=dict(map(reversed,ind2note.items()))

# Timestep

#### Every x notes will output one note

In [53]:
#timestep
timesteps=30

#store values of input and output
x=[] ; y=[]

for i in new_notes:
 for j in range(0,len(i)-timesteps):
  #input will be the current index + timestep
  #output will be the next index after timestep
  inp=i[j:j+timesteps] ; out=i[j+timesteps]

  #append the index value of respective notes
  x.append(list(map(lambda x:note2ind[x],inp)))
  y.append(note2ind[out])

x_new=np.array(x)
y_new=np.array(y)

In [54]:
#reshape input and output for the model
x_new = np.reshape(x_new,(len(x_new),timesteps,1))
y_new = np.reshape(y_new,(-1,1))

#split the input and value into training and testing sets
#80% for training and 20% for testing sets
x_train,x_test,y_train,y_test = train_test_split(x_new,y_new,test_size=0.2,random_state=42)


In [55]:
#create the model
model = Sequential()
#create two stacked LSTM layer with the latent dimension of 256
model.add(LSTM(128,return_sequences=True,input_shape=(x_new.shape[1],x_new.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
#fully connected layer for the output with softmax activation
model.add(Dense(len(note2ind),activation='softmax'))
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_8 (LSTM)                        │ (None, 30, 128)             │          66,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 30, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 65)                  │           8,385 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 223,041 (871.25 KB)

 Trainable params: 223,041 (871.25 KB)

 Non-trainable params: 0 (0.00 B)

# Model Training

In [56]:
#compile the model using Adam optimizer
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

#train the model on training sets and validate on testing sets
model.fit(
 x_train,y_train,
 batch_size=128,epochs=120,
 validation_data=(x_test,y_test))

Epoch 1/120
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.0142 - loss: 4.1645 - val_accuracy: 0.0000e+00 - val_loss: 4.0604
Epoch 2/120
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - accuracy: 0.0699 - loss: 4.0189 - val_accuracy: 0.0000e+00 - val_loss: 4.0114
Epoch 3/120
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 212ms/step - accuracy: 0.0738 - loss: 3.9000 - val_accuracy: 0.0000e+00 - val_loss: 3.9715
Epoch 4/120
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 224ms/step - accuracy: 0.0557 - loss: 3.8012 - val_accuracy: 0.0000e+00 - val_loss: 3.9400
Epoch 5/120
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 216ms/step - accuracy: 0.0764 - loss: 3.6879 - val_accuracy: 0.0227 - val_loss: 3.9210
Epoch 6/120
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step - accuracy: 0.0946 - loss: 3.5902 - val_accuracy: 0.0227 - val_loss: 3.8701
Epoch 7/120
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step - accuracy: 0.1126 - loss: 3.5089 - val_accuracy: 0.0909 - val_loss: 3.8283
Epoch 8/120
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.1438 - loss: 3.4107 - val_accurac

# Save the data

In [57]:
#save the model for predictions
model.save("model.keras")

# Composition

In [58]:
#load the model
model = load_model("model.keras")
#generate random index
index = np.random.randint(0,len(x_test)-1)
#get the data of generated index from x_test
music_pattern = x_test[index]
out_pred=[] #it will store predicted notes

#iterate till 200 note is generated
for i in range(100):

 #reshape the music pattern
 music_pattern = music_pattern.reshape(1,len(music_pattern),1)

 #get the maximum probability value from the predicted output
 pred_index = np.argmax(model.predict(music_pattern))
 #get the note using predicted index and
 #append to the output prediction list
 out_pred.append(ind2note[pred_index])
 music_pattern = np.append(music_pattern,pred_index)

 #update the music pattern with one timestep ahead
 music_pattern = music_pattern[1:]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103m

# Saving the midi file

In [59]:
output_notes = []
# Print out_pred to check predicted notes
print("Predicted Notes (out_pred):", out_pred)

try:
    for offset, pattern in enumerate(out_pred):
        # Process chords (notes separated by '.') or single notes
        if '.' in pattern or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            parsed_notes = []

            # Parse all notes in the chord
            for current_note in notes_in_chord:
                if current_note.isdigit():
                    parsed_note = note.Note(int(current_note))
                else:
                    parsed_note = note.Note(current_note)
                parsed_notes.append(parsed_note)

            # Sort notes to identify the bass (lowest) note
            parsed_notes.sort(key=lambda x: x.pitch.midi)
            bass_note = parsed_notes[0]

            # Create bass note with long duration (whole note)
            bass_note.storedInstrument = instrument.Piano()
            bass_note.duration = duration.Duration(10.0)
            bass_note.offset = offset
            output_notes.append(bass_note)

            # Create chord with remaining notes (shorter duration)
            remaining_notes = parsed_notes[1:]
            if remaining_notes:
                higher_chord = chord.Chord(remaining_notes)
                higher_chord.storedInstrument = instrument.Piano()
                higher_chord.duration = duration.Duration(1.0)  # Quarter note
                higher_chord.offset = offset
                output_notes.append(higher_chord)

        else:  # Single note processing
            if pattern.isdigit():
                new_note = note.Note(int(pattern))
            else:
                new_note = note.Note(pattern)

            # Treat single notes as bass notes with long duration
            new_note.storedInstrument = instrument.Piano()
            new_note.duration = duration.Duration(4.0)  # Whole note
            new_note.offset = offset
            output_notes.append(new_note)

except Exception as e:
    print(f"Error during note creation: {e}")

# Print output_notes to check generated notes
print("Generated Notes (output_notes):", output_notes)

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='/content/drive/MyDrive/test_output.mid')

# For Google Colab, download the file
from google.colab import files
files.download('/content/drive/MyDrive/test_output.mid')

Predicted Notes (out_pred): ['G#5', 'G#5', 'G#5', 'G#5', 'E-3', 'E-5.F5', 'E-5', 'C#5', 'E-3', 'C5', 'C5', 'G#3', 'G#4', 'C5', 'C5', 'C5', 'C#5.G#3', 'C#5.G#3', 'F5.B-2', 'F5.B-2', 'B-4.C5', 'B-2', 'B-2', 'F5', 'G#5', 'G#5', 'G#5', 'G#5', 'G#5', 'G#5', 'E-3', 'E-5.F5', 'E-5', 'C#5', 'C5', 'C5', 'C5', 'G#3', 'G#3', 'C5', 'C5', 'G#3', 'C5', 'C#5.G#3', 'F5.B-2', 'F5.B-2', 'B-4.C5', 'B-2', 'B-2', 'F5', 'G#5', 'G#5', 'G#5', 'G#5', 'G#5', 'G#5', 'E-3', 'E-5.F5', 'E-5.F5', 'E-5', 'E-3', 'C5', 'C5', 'G#3', 'G#3', 'C5', 'C5', 'C5', 'G#3', 'C#5.G#3', 'F5.B-2', 'F5.B-2', 'C#5', 'B-2', 'B-2', 'F5', 'G#5', 'G#5', 'G#5', 'G#5', 'G#5', 'G#5', 'E-3', 'E-5.F5', 'E-5.F5', 'E-5', 'E-3', 'C5', 'C5', 'G#3', 'G#4', 'C5', 'C5', 'C5', 'C5', 'C#5.G#3', 'F5.B-2', 'F5.B-2', 'B-4.C5', 'B-2']
Generated Notes (output_notes): [<music21.note.Note G#>, <music21.note.Note G#>, <music21.note.Note G#>, <music21.note.Note G#>, <music21.note.Note E->, <music21.note.Note E->, <music21.chord.Chord F5>, <music21.note.Note E->

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>